# 1.EDA
*** 


<a id="12"></a> 
# 1.1 IMPORT MODULES

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV,RidgeCV,ElasticNetCV,LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score,GridSearchCV
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import mean_squared_error
import warnings
from scipy.stats import  skew


In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)


In [ ]:
train = pd.read_csv("/kaggle/input/regression-technique-eda/House Price Regression Tecnique/train.csv")
test = pd.read_csv("/kaggle/input/regression-technique-eda/House Price Regression Tecnique/test.csv")
df = train.append(test,ignore_index=False).reset_index()
df = df.drop("index", axis=1)

<a id="12"></a> 
# 1.2 GENERAL INFO OF DATA SET

In [ ]:
def dataframe_info(df):
    print("Info")
    print(df.info())
    print("#"*40)
    print("DESCRIPTION")
    print(df.describe())
    print("*"*40)
    print("NAN VALUES")
    print(df.isna().sum())
    print("*"*40)
    print("# OF UNIQUES")
    print(df.nunique())
    print("*"*40)
    print("# OF DUPLICATED")
    print(df.duplicated().sum())
    print("*"*40)
    print("UNIQUES")
    for i in df.columns:
        if  df[i].nunique()>20:
            print("Contains more than 20 unique value.")
            print(i,":",df[i].nunique(),df[i].unique()[:20],end="\n\n\n")
        else:
            print("Contains less than 20 unique value.")
            print(i,":",df[i].nunique(),df[i].unique()[:20],end="\n\n\n")

        
dataframe_info(df)


In this function,i will categorized numerical columns, categorical columns. The reason of this function is that some numerical values look like categorical. Let me expalin. In this data set, OverallQuall looks like numerical. If you check closely the data, each numebrs are categorical.

OverallQuall:

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor

<div class="alert alert-block alert-success">
<b>Grab_col_names:</b>  It gives the names of categorical, numerical and categorical but cardinal variables in the data set.
</div>


In [ ]:
def grab_col_names(dataframe,cat_th=16,car_th=20):
    """
    It gives the names of categorical, numerical and categorical but cardinal variables in the data set.
    Note: Categorical variables with numerical appearance are also included in categorical variables.

    Parameters
    ------
            df: Dataframe
                The dataframe from which variable names are to be retrieved
        cat_th: int, optional
                threshold value for numeric but categorical variables
        car_th: int, optinal
                threshold value for categorical but cardinal variables

    Returns
    ------
        cat_cols: list
                Categorical variable list
        num_cols: list
                Numeric variable list
        cat_but_car: list
                Categorical but cardinal variable list

    Notes
    ------
        cat_cols + num_cols + cat_but_car = total number of variables
        num_but_cat is inside cat_cols
   """
     #Category & cardinal columns
    cat_cols =[col for col in df.columns if df[col].dtypes=="O"]
    num_but_cats = [col for col in df.columns if (df[col].nunique()<cat_th) and df[col].dtypes != "O"]
    cat_but_car = [col for col in df.columns if (df[col].nunique()>car_th) and df[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cats
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    #Numerical columns
    num_cols = [col for col in df.columns if df[col].dtypes !="O"]
    num_cols = [col for col in num_cols if col not in num_but_cats]

    print(f"Observations: {df.shape[0]}")
    print(f"Variables: {df.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cats)}')
    return cat_cols, num_cols, cat_but_car


In [ ]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)


# 2. HANDLING OUTLIERS
***

**Tresholds Control & Rearrange it.**

In [ ]:
def outlier_tresholds(df,col_name,qu1=0.05,qu3=0.95):
    q1 = df[col_name].quantile(qu1)
    q3 = df[col_name].quantile(qu3)
    iqr=q3-q1
    lower = q1-1.5*iqr
    upper = q3+1.5*iqr
    return lower,upper

def check_outlier(df,col_name):
    lower_bond, upper_bond = outlier_tresholds(df,col_name)

    if df[(df[col_name]<lower_bond) | (df[col_name]>upper_bond)].any(axis=None):
        return True
    else:
        return False


outliers=[]
for i in num_cols:
    if check_outlier(df,i):
        outliers.append(i)
print(outliers)

def check_outlier_cat(df,col_name):
        upper_0 = []
        for i in col_name:
            lower, upper = outlier_tresholds(df, i)
            print(i)
            print(df[df[i]>upper]["SalePrice"].nunique())
            print("OUTLIER:")
            print("UPPER",upper)
            print(df[df[i]>upper][i])
            print(df[i].head())
            print("**"*20)

            if upper==0.0:
                upper_0.append(i)
        return upper_0

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_tresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit


In [ ]:
for i in outliers:
    replace_with_thresholds(df,i)

check_outlier_cat(df,outliers)


for i in outliers:
    print(i,check_outlier(df,i))

#From now on, there is no treshold.

# Work on Missing Values
null_columns = []


for i in df.columns:
   if (df[i].isna()>0).any(axis=None):
       null_columns.append(i)



# 3. MISSING VALUES
***

# 3.1 CATEGRORIZE NULL VALUES

<div class="alert alert-block alert-danger"> 
<b>BE CAREFUL:</b> According yo data_description, BsmtQual,BsmtCond, BsmtExposure,BsmtFinType1,BsmtFinType2,FireplaceQu,GarageType,GarageFinish, BsmtCond
GarageQual,GarageCond,PoolQC,Fence,MiscFeature,GarageYrBlt,MasVnrType,MasVnrArea. NA values mean NO. For exmaple, BsmtCond: NA means No Basement. Also, some of them have direct realtion btw them.
For example, GarageType has na values. It means no garage. Also, Data contains GarageYrBlt. It has na values. Those na means no garage.Then, i will create another list for them and rearange null_columns.
</div>


In [ ]:
null_but_not = ["BsmtQual", "BsmtExposure","BsmtFinType1","BsmtFinType2","FireplaceQu","GarageType","GarageFinish", "GarageQual","GarageCond","PoolQC","Fence",
                "MiscFeature","Alley","GarageYrBlt","BsmtCond","MasVnrType","MasVnrArea","SalePrice","Neighborhood"]

null_columns = [col for col in null_columns if col not in null_but_not]

**Before filling the NA values, i will check the relation btw na columns and target columns if NA value is random or not.**

In [ ]:
def missing_vs_target(df,target,na_columns):
    temp_df = df.copy()

    for col in na_columns:
        temp_df[col +"_NA_FLAG"] = np.where(temp_df[col].isnull(),1,0)

    na_flags = temp_df.loc[:,temp_df.columns.str.contains("_NA_")].columns

    for col in na_flags:
        print(pd.DataFrame({"TARGET_MEAN":temp_df.groupby(col)[target].mean(),
                            "Count": temp_df.groupby(col)[target].count()}), end="\n\n\n")


In [ ]:
missing_vs_target(df,"SalePrice",null_columns)


### Let's see null grade graph.

In [ ]:
def na_columns_histogram(df,na_cols):
    for col in na_cols:
        df[col].hist()
        plt.title("Variable names: {}, Variable type: {} and # of unique:{}".format(col,df[col].dtypes,df[col].nunique()))
        plt.xlabel("Total NA values: {}".format(df[col].isna().sum()))
        plt.grid(False)
        plt.show(block=True)

In [ ]:
na_columns_histogram(df,null_columns)

# 3.2 FILL NA VALUES

In [ ]:
num_null_cols = [col for col in null_columns if df[col].dtypes != "O"]

for i in num_null_cols:
    df[i] = df[i].fillna(df[i].median())

print("numerical null features",df[num_null_cols].isna().sum())

cat_null_cols = [col for col in null_columns if df[col].dtypes =="O"]

for i in cat_null_cols:
    df[i] = df[i].fillna(df[i].mode()[0])
print("categorical null features",df[cat_null_cols].isna().sum())

# 4. Rare Encoding
***

In [ ]:

def rare_encoder(dataframe, rare_perc):
    temp_df = dataframe.copy()

    rare_columns = [col for col in temp_df.columns if temp_df[col].dtypes == 'O'
                    and (temp_df[col].value_counts() / len(temp_df) < rare_perc).any(axis=None)]

    for var in rare_columns:
        tmp = temp_df[var].value_counts() / len(temp_df)
        rare_labels = tmp[tmp < rare_perc].index
        temp_df[var] = np.where(temp_df[var].isin(rare_labels), 'Rare', temp_df[var])

    return temp_df

In [ ]:
df = rare_encoder(df,0.011)


# 5. FEATURE SELECTION - 1
***

In [ ]:
df.isna().sum()


<div class="alert alert-block alert-warning">
<b>NOTE:</b> As i explained above, fill na values with No. Also, let's find out numerical and categorical columns
</div>

In [ ]:
na_num=[col for col in df.columns if df[col].dtypes != "O" and (df[col].isna().sum()>0).any(axis=None)]
na_cat=[col for col in df.columns if df[col].dtypes == "O" and (df[col].isna().sum()>0).any(axis=None)]


df[na_cat]=df[na_cat].apply(lambda x: np.where(x.isna(),"No",x))

In [ ]:
drop_list = []
for i in df.columns:
    temp=pd.DataFrame({i: df[i].value_counts(),
                  "Mean": np.round(df.groupby(i)["SalePrice"].mean(), 0),
                  "Ratio": df[i].value_counts() / len(df[i])})
    if (temp[temp.Ratio>0.95]).any(axis=None):
        drop_list.append(i)


<div class="alert alert-block alert-danger"> 
<b>BE CAREFUL:</b> Neighborhood feature has high cardinality, thus it is added to drop list.
</div>


In [ ]:
drop_list.append("Neighborhood")

In [ ]:
df.drop(drop_list,axis=1,inplace=True)


Again categorise the numerical columns and categorical columns.

# 6. FEATURE GENERATION
***

In [ ]:

df["NEW_1st*GrLiv"] = df["1stFlrSF"] * df["GrLivArea"]

df["NEW_Garage*GrLiv"] = (df["GarageArea"] * df["GrLivArea"])


# Total Floor
df["NEW_TotalFlrSF"] = df["1stFlrSF"] + df["2ndFlrSF"] # 32

# Total Finished Basement Area
df["NEW_TotalBsmtFin"] = df.BsmtFinSF1 + df.BsmtFinSF2 # 56

# Porch Area
df["NEW_PorchArea"] = df.OpenPorchSF + df.EnclosedPorch + df.ScreenPorch +  df.WoodDeckSF # 93

# Total House Area
df["NEW_TotalHouseArea"] = df.NEW_TotalFlrSF + df.TotalBsmtSF # 156

df["NEW_TotalSqFeet"] = df.GrLivArea + df.TotalBsmtSF # 35


# Lot Ratio
df["NEW_LotRatio"] = df.GrLivArea / df.LotArea # 64

df["NEW_RatioArea"] = df.NEW_TotalHouseArea / df.LotArea # 57

df["NEW_GarageLotRatio"] = df.GarageArea / df.LotArea # 69

# MasVnrArea
df["NEW_MasVnrRatio"] = df.MasVnrArea / df.NEW_TotalHouseArea # 36

# Dif Area
df["NEW_DifArea"] = (df.LotArea - df["1stFlrSF"] - df.GarageArea - df.NEW_PorchArea - df.WoodDeckSF) # 73


df["NEW_OverallGrade"] = df["OverallQual"] * df["OverallCond"] # 61


df["NEW_Restoration"] = df.YearRemodAdd - df.YearBuilt # 31

df["NEW_HouseAge"] = df.YrSold - df.YearBuilt # 73

df["NEW_RestorationAge"] = df.YrSold - df.YearRemodAdd # 40

df["NEW_GarageAge"] = df.GarageYrBlt - df.YearBuilt # 17

df["NEW_GarageRestorationAge"] = np.abs(df.GarageYrBlt - df.YearRemodAdd) # 30

df["NEW_GarageSold"] = df.YrSold - df.GarageYrBlt # 48


In [ ]:

cat_cols_3, num_cols_3, cat_but_car = grab_col_names(df)
num_cols_3 = [col for col in num_cols_3 if col not in ["SalePrice","Id"]]

df[num_cols_3].isna().sum()


# 6.1 Handling New Missing Values

In [ ]:
df[cat_cols_3]=df[cat_cols_3].apply(lambda x: np.where(x.isna(),"No",x))
def quick_missing_imp(data, num_method="median", cat_length=20, target="SalePrice"):
    variables_with_na = [col for col in data.columns if data[col].isnull().sum() > 0]  # Eksik değere sahip olan değişkenler listelenir

    temp_target = data[target]

    print("# BEFORE")
    print(data[variables_with_na].isnull().sum(), "\n\n")  # Uygulama öncesi değişkenlerin eksik değerlerinin sayısı

    # değişken object ve sınıf sayısı cat_lengthe eşit veya altındaysa boş değerleri mode ile doldur
    data = data.apply(lambda x: x.fillna(x.mode()[0]) if (x.dtype == "O" and len(x.unique()) <= cat_length) else x, axis=0)

    # num_method mean ise tipi object olmayan değişkenlerin boş değerleri ortalama ile dolduruluyor
    if num_method == "mean":
        data = data.apply(lambda x: x.fillna(x.mean()) if x.dtype != "O" else x, axis=0)
    # num_method median ise tipi object olmayan değişkenlerin boş değerleri ortalama ile dolduruluyor
    elif num_method == "median":
        data = data.apply(lambda x: x.fillna(x.median()) if x.dtype != "O" else x, axis=0)

    data[target] = temp_target

    print("# AFTER \n Imputation method is 'MODE' for categorical variables!")
    print(" Imputation method is '" + num_method.upper() + "' for numeric variables! \n")
    print(data[variables_with_na].isnull().sum(), "\n\n")

    return data

In [ ]:

df = quick_missing_imp(df,num_method="median",cat_length=17)

df.isna().sum()

# 7. TRANSFORMATION AND SCALING
***

<div class="alert alert-block alert-success">
<b>GENERAL INFO:</b> Scaling, Standardizing and Transformation are important steps of numeric feature engineering and they are being used to treat skewed features and rescale them for modelling.
</div>


In [ ]:
def plot_data(df,col_name):
    import scipy.stats as stat
    import pylab
    plt.figure(figsize=(10,8))
    plt.subplot(1,2,1)
    sns.histplot(df[col_name],kde=True)
    plt.subplot(1,2,2)
    stat.probplot(df[col_name],dist="norm", plot=pylab)
    plt.show(block=True)

def log_transformation(df,col_name,plot=False):
    temp_df = df.copy()
    temp_df[col_name+"_log"]=np.log(temp_df[col_name]+1)
    if plot:
        plot_data(temp_df,col_name+"_log")
    return skew(temp_df[col_name+"_log"])

def reciprocal_transformation(df,col_name,plot=False):
    temp_df = df.copy()
    temp_df[col_name+"_reciprocal"]=1/(temp_df[col_name])
    if plot:
        plot_data(temp_df,col_name+"_reciprocal")
    return skew(temp_df[col_name+"_reciprocal"])

def square_root_transformation(df,col_name,plot=False):
    temp_df = df.copy()
    temp_df[col_name+"_square"]=temp_df[col_name]**(1/2)

    if plot:
        plot_data(temp_df,col_name+"_square")
    return skew(temp_df[col_name+"_square"])

def zscore_transformation(df,col_name,plot=False):
    from scipy.stats import zscore
    temp_df = df.copy()
    temp_df[col_name+"_zscore"]=zscore(temp_df[col_name])

    if plot:
        plot_data(temp_df,col_name+"_zscore")
    return skew(temp_df[col_name+"_zscore"])

def quantile_transform(df,col_name,out_dist = "uniform",n_q=1000,plot=False):
    """
    Parameters
    ----------
    df: pd.Dataframe()
        Dataframe
    col_name: string
        Column name which you wantedd to apply this fuction
    out_dist:{‘uniform’, ‘normal’}, default=’uniform’
    Marginal distribution for the transformed data. The choices are ‘uniform’ (default) or ‘normal’.
    n_q:n_quantiles int, default=1000 or n_samples

    plot Bool, default=False
        Generates a probability plot of sample data against the quantiles of a specified theoretical distribution (the normal distribution by default)

    Returns
    -------
    Compute the sample skewness of a data set.
    """
    from sklearn.preprocessing import QuantileTransformer
    temp_df=df.copy()
    quantile = QuantileTransformer(output_distribution=out_dist,n_quantiles=n_q)
    temp_df[col_name+"_quant"] = quantile.fit_transform(df[[col_name]].values)
    if plot:
        plot_data(temp_df,col_name+"_quant")
    return skew(temp_df[col_name+"_quant"])

transform_list = [log_transformation,reciprocal_transformation,square_root_transformation,zscore_transformation,quantile_transform]


In [ ]:
for i in transform_list:
    print(i)
    for j in num_cols_3:

        print(f"{j} transform. Skew value:{i(df,j)}")
    print("*"*40)

<div class="alert alert-block alert-success">
<b>RESULT: </b> Pls, use which one is proper your data. When i compared the skew values, quantile transformation is the smallest value.
It means the columns get closer to the normal distribution
</div>

In [ ]:

def apply_quantile(df,col_name):
    from sklearn.preprocessing import QuantileTransformer
    quantile = QuantileTransformer(output_distribution="uniform")
    df[col_name] = quantile.fit_transform(df[[col_name]].values)
    return skew(df[col_name])

for i in num_cols_3:
    apply_quantile(df,i)


# 8. ENCODING
***

# 8.1 LABEL ENCODING

In [ ]:
def label_encoder(dataframe, binary_col):
    from sklearn.preprocessing import LabelEncoder
    labelencoder = LabelEncoder()
    dataframe[binary_col] = labelencoder.fit_transform(dataframe[binary_col])
    return dataframe

binary_cols = [col for col in df.columns if df[col].dtypes == "O" and len(df[col].unique()) == 2]

for col in binary_cols:
    label_encoder(df, col)

# 8.2 ONE HOT ENCODING

In [ ]:

def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe

df = one_hot_encoder(df, cat_cols_3, drop_first=True)

*Feature Selection, In this part, we ll select features by using tree-based feature selection method. Firstly, I will select a feature number by comparing LassoCV, RidgeCV & ElasticCV. Those models give the minumum number of feature numbers for ml. For example, in this project, The smallest number of feaure is selected by LassoCV. I will select this method. I will combine the advice of multiple, different, models to decide on which features are worth keeping. Firstly i selected LassoCV & i will choose tow models from RandomForestRegressor,LGBMRegressor,XGBRegressor, AdaBoostRegressor & GradientBoostingRegressor*

In [ ]:


def split_model (x,y,random_s= 42, test_s=0.3):

    x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=random_s,test_size=test_s)

    return x_train,x_test,y_train,y_test


def number_of_feature_selection(x,y,Lasso = False,Ridge =False, Elatic =False):
    """

    Parameters
    ----------
    x: Features
    y: Target feature
    Lasso: LassoCV
    Ridge: RidgeCV
    Elatic: ElasticNetCV

    Returns
    -------
    Model names and min requirements for best modelling.
    And mask list of each of three model.

    """
    x_train, x_test, y_train, y_test = split_model(x,y)

    linear_CV = [LassoCV,RidgeCV,ElasticNetCV]
    n_feature = []

    mask_liste = []

    if Lasso:
        reg = LassoCV(cv=5).fit(X=x_train, y=y_train)
        mask = reg.coef_ != 0
        mask_liste.append(mask)
        print(mask.sum())
        liste = []
        reduced_x = x.loc[:, mask].columns
        liste.append(reduced_x)
        n_feature.append(liste)
    elif Ridge:
        reg = RidgeCV(cv=5).fit(X=x_train, y=y_train)
        mask = reg.coef_ != 0
        mask_liste.append(mask)
        print(mask.sum())
        liste = []
        reduced_x = x.loc[:, mask].columns
        liste.append(reduced_x)
        n_feature.append(liste)
    elif Elatic:
        reg = ElasticNetCV(cv=5).fit(X=x_train, y=y_train)
        mask = reg.coef_ != 0
        mask_liste.append(mask)
        print(mask.sum())
        liste = []
        reduced_x = x.loc[:, mask].columns
        liste.append(reduced_x)
        n_feature.append(liste)

    else:
        for i in linear_CV:

            print(i)
            reg = i(cv=5).fit(X=x_train, y=y_train)
            mask = reg.coef_ != 0
            mask_liste.append(mask)
            print(mask.sum())
            liste = []
            reduced_x = x.loc[:, mask].columns
            liste.append(reduced_x)
            n_feature.append(liste)

    return n_feature,mask_liste



In [ ]:
train_df = df[df['SalePrice'].notnull()]
test_df = df[df['SalePrice'].isnull()]

x = train_df.drop(["Id","SalePrice"],axis=1)
y = train_df["SalePrice"]
y_ = train_df[["SalePrice"]]

<div class="alert alert-block alert-success">
<b>RESULT: </b> By using LassoCV, RidgeCV & ElasticCV, we selected min feature requirements for model. Thus, i will select Lasso for combining moodels to select features which are worth keeping.
</div>


# 9. FEATURE SELECTION-2
***

In [ ]:
model_liste = [GradientBoostingRegressor(), AdaBoostRegressor(), XGBRegressor(),LGBMRegressor(),RandomForestRegressor()]
def feature_selection(x,y,model_liste,limitation=2,step_ = 20):
    """

    Parameters
    ----------
    x: Features
    y: Target feature
    model_liste: list of model
    limitation: In the function, 3 models used. If 2/3 of the models choose the features. It wll be kept.
    step_: If greater than or equal to 1, then step corresponds to the (integer) number of features to remove at each iteration.
    If within (0.0, 1.0), then step corresponds to the percentage (rounded down) of features to remove at each iteration.

    Returns
    -------
    DataFrame of three based model (combined models to select feature) with r2 and number of features.
    Reduced list of each  three based model.
    Dataframe of mask list
    """
    liste,mask_liste = number_of_feature_selection(x,y,Lasso=True)

    models = model_liste
    final_result = []
    reduced_x_list = []

    j=0
    while j < len(models):
        for k in models[j:]:
            if k != models[j]:
                x_train, x_test, y_train, y_test = split_model(x, y)
                print("Model:", models[j], ",", k)
                reg_1 = RFE(estimator=models[j],
                            n_features_to_select=len(liste[0][0]),
                            step=step_,
                            verbose=1)
                reg_1.fit(x_train, y_train)
                mask_1 = reg_1.support_

                reg_2 = RFE(estimator=k,
                            n_features_to_select=len(liste[0][0]),
                            step=step_,
                            verbose=1)
                reg_2.fit(x_train, y_train)
                mask_2 = reg_2.support_
                mask_0 = mask_liste[0]

                votes = np.sum([mask_0, mask_1, mask_2], axis=0)

                masked = votes >= limitation

                reduced_x = x.loc[:, masked]

                x_train, x_test, y_train, y_test = split_model(reduced_x, y)

                le = LinearRegression()
                le.fit(x_train, y_train)
                r_squared = le.score(x_test, y_test)
                print(
                    f'The model can explain {r_squared:.1%} of the variance in the test set using {len(le.coef_)} features.')
                reduced_x_list.append([models[j],k,reduced_x])
                final_result.append([models[j], k, r_squared, len(le.coef_)])

        j += 1

    df = pd.DataFrame(final_result,columns=["model_1","model_2","r2","number_of_feature"])
    df_reduced_x =pd.DataFrame(reduced_x,columns=reduced_x.columns)

    return df,reduced_x_list,df_reduced_x


In [ ]:
feature_model,reduced_x_list,df_reduced_x = feature_selection(x,y,model_liste,limitation=2, step_=20)


In [ ]:
feature_model

<div class="alert alert-block alert-success">
<b>RESULT: </b> Selectting features by using AdaBoostRegressor() & GradientBoostingRegressor(). Feature number is dropped from 254 to 60.
</div>

In [ ]:
feature_model,reduced_x_list,df_reduced_x = feature_selection(x,y,[RandomForestRegressor(), XGBRegressor()],limitation=2, step_=20)


# 9.1 MASK TEST & TRAIN DATA SET

In [ ]:
mask = df_reduced_x.columns

In [ ]:
test_df=test_df[mask]
train_df = train_df[mask]


In [ ]:
reduced_df_wo_saleprice= pd.concat([train_df,test_df])
reduced_df = pd.concat([reduced_df_wo_saleprice,y],axis=1)

# 10. DIMENSIONALITY REDUCTION
***

In [ ]:
train_df = reduced_df[reduced_df['SalePrice'].notnull()]
test_df = reduced_df[reduced_df['SalePrice'].isnull()]



In [ ]:
y = train_df['SalePrice']
y_= train_df[['SalePrice']]
x = train_df.drop(["SalePrice"], axis=1)

In [ ]:
y.shape

# 10.1 PCA DECOMPOSITION

In [ ]:
x_train,x_test,y_train,y_test = split_model(x,y)

In [ ]:
pca = PCA(n_components=0.991)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)


In [ ]:
len(x_train_pca[0])

Reduced features by PCA Decomposition. Feature number is reduced from 66 to 49.

# 11. MODELLING
***

In [ ]:
models = [('KNN', KNeighborsRegressor()),
          ('GBM', GradientBoostingRegressor()),
          ("XGBoost", XGBRegressor(objective='reg:squarederror')),
          ("LightGBM", LGBMRegressor()),
          ("AdaBoost",AdaBoostRegressor()),
          ("CatBoost", CatBoostRegressor(verbose=False))]


In [ ]:
for name, regressor in models:
    RMSE = np.mean(np.sqrt(-cross_val_score(regressor, df_reduced_x, y, cv=5, scoring="neg_mean_squared_error")))
    print(f"R2: {round(RMSE, 4)} ({name})")

In [ ]:
for name, regressor in models:
    r2 = np.mean(cross_val_score(regressor, df_reduced_x, y, cv=5, scoring="r2"))
    print(f"R2: {round(r2, 4)} ({name})")


<div class="alert alert-block alert-success">
<b>RESULT: </b> Best model for this data set is that CatBoostRegression.
</div>

# 12 .OPTIMIZATION
***

# 12.1 Modelling with PCA

In [ ]:

catboost_params = {"iterations": [1000,2000],
                   "learning_rate": [0.01,0.2]}
catboost_model = CatBoostRegressor(random_state=46)

catboost_gradient = GridSearchCV(catboost_model,
                            catboost_params,
                            cv=5,
                            verbose=True,
                            n_jobs=-1
                            ).fit(x_train_pca,y_train)


In [ ]:
catboost_gradient.best_params_

In [ ]:
cb_model_pca = CatBoostRegressor(**catboost_gradient.best_params_).fit(x_train_pca,y_train)

In [ ]:
cb_rmse_pca = np.mean(np.sqrt(-cross_val_score(cb_model_pca, x_train_pca, y_train, cv=5, scoring="neg_mean_squared_error")))

In [ ]:
cb_rmse_pca

# 12.2 Modelling without PCA

In [ ]:
cb_model_wo_pca = CatBoostRegressor(**catboost_gradient.best_params_).fit(x_train,y_train)

cb_rmse_wo_pca = np.mean(np.sqrt(-cross_val_score(cb_model_wo_pca, x_train_pca, y_train, cv=5, scoring="neg_mean_squared_error")))

In [ ]:
cb_rmse_wo_pca 

<div class="alert alert-block alert-success">
<b>RESULT: </b> When comparing catboost regression with pca decomposition and without it, their rmse values are almost the same. Thus, it will be good preferance to work with reduced by PCA.
</div>

# 13. FEATURE IMPORTANCE
***

In [ ]:
colu = x_train.columns[:len(x_train_pca[0])]
colu

In [ ]:
feature_imp = pd.DataFrame({"Value": cb_model_pca.feature_importances_, "Feature": colu})
plt.figure(figsize=(10, 10))
sns.set(font_scale=1)
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[0:10])
plt.title("Features")
plt.tight_layout()
plt.show(block=True)
